<a href="https://colab.research.google.com/github/ADorantesCR/resumenautomatico/blob/main/sintesisfinalviernes.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [148]:
!pip install pandas python-docx
!pip install Pillow


In [149]:
###########semarnat ANÁLISIS MÁS ÍNDICE#

import os
import pandas as pd
import nltk
import re
from nltk.corpus import stopwords
from docx import Document
from docx.shared import Pt
from docx.enum.text import WD_PARAGRAPH_ALIGNMENT

# Ruta de la carpeta que contiene los archivos Excel
ruta = '/content/drive/MyDrive/profepa-sintesis/Excel/SEMARNAT'

# Obtener la lista de archivos Excel en la carpeta
archivos_excel = [archivo for archivo in os.listdir(ruta) if archivo.endswith('.xlsx')]

# DataFrame final
df_finalS = pd.DataFrame()

# Recorrer los archivos Excel y cargarlos en el DataFrame final
for i, archivo in enumerate(archivos_excel):
    archivo_path = os.path.join(ruta, archivo)
    df = pd.read_excel(archivo_path)

    # Eliminar las filas correspondientes según el archivo
    if i == 0:
        df = df.iloc[1:]  # Eliminar la primera fila
    else:
        df = df.iloc[2:]  # Eliminar las tres primeras filas

    # Agregar el DataFrame al DataFrame final
    df_finalS = pd.concat([df_finalS, df], ignore_index=True)

# Utilizar la primera fila como cabeceras
df_finalS.columns = df_finalS.iloc[0]
df_finalS = df_finalS.iloc[1:]

# Convertimos las columnas 'Fecha' y 'Hora' a datetime
df_finalS['Fecha'] = pd.to_datetime(df_finalS['Fecha'])
df_finalS['Hora'] = pd.to_timedelta(df_finalS['Hora'])

# Creamos una nueva columna con la fecha y hora juntas
df_finalS['FechaHora'] = df_finalS['Fecha'] + df_finalS['Hora']

# Definimos la fecha y hora de inicio y fin
fecha_inicio = pd.to_datetime('2023-06-29 17:00:00')
fecha_fin = pd.to_datetime('2023-06-30 06:00:00')

# Filtramos el dataframe para que solo queden las filas dentro del rango de fecha y hora
df_finalS = df_finalS[(df_finalS['FechaHora'] >= fecha_inicio) & (df_finalS['FechaHora'] <= fecha_fin)]

#copiamos la columan titulo para trabajar sobre ella
df_finalS['Título_analisis'] = df_finalS['Título']

# Descargar los stopwords si no están descargados
nltk.download('stopwords')

# Obtener los stopwords en español
stopwords_esp = set(stopwords.words('spanish'))

# Función para eliminar palabras vacías y manejar valores nulos
def eliminar_palabras_vacias(texto):
    if isinstance(texto, str):  # Verificar si el valor no es nulo
        texto = re.sub(r'[^\w\s]', '', texto)  # Eliminar caracteres no deseados utilizando re.sub()
        texto = texto.strip()  # Eliminar espacios en blanco adicionales al inicio y al final
        palabras = texto.split()
        palabras_filtradas = [palabra for palabra in palabras if palabra not in stopwords_esp]
        return ' '.join(palabras_filtradas).lower()
    else:
        return texto

# Aplicar la función a la columna 'Título_analisis'
df_finalS['Título_analisis'] = df_finalS['Título_analisis'].apply(eliminar_palabras_vacias)

# Verificar y convertir a cadena los valores no nulos en la columna "Título_analisis"
df_finalS['Título_analisis'] = df_finalS['Título_analisis'].apply(lambda x: str(x) if pd.notnull(x) else '')

grupos = {}
for i in range(len(df_finalS)):
    titulo_i = df_finalS.iloc[i, df_finalS.columns.get_loc("Título_analisis")]
    if i not in grupos:
        grupo_actual = len(grupos) + 1
        grupos[i] = grupo_actual
    else:
        grupo_actual = grupos[i]
    for j in range(i+1, len(df_finalS)):
        titulo_j = df_finalS.iloc[j, df_finalS.columns.get_loc("Título_analisis")]
        titulo_j = str(titulo_j) if pd.notnull(titulo_j) else ''
        palabras_iguales = set(titulo_i.lower().split()) & set(titulo_j.lower().split())
        if len(palabras_iguales) >= 3:
            if j not in grupos:
                grupos[j] = grupo_actual
            else:
                grupo_j = grupos[j]
                if grupo_j != grupo_actual:
                    for k, v in grupos.items():
                        if v == grupo_j:
                            grupos[k] = grupo_actual

# Agregar una columna al DataFrame para mostrar los grupos
df_finalS['Grupo'] = [grupos[i] for i in range(len(df_finalS))]

grupo_counts = df_finalS['Grupo'].value_counts()
df_finalS['Notas'] = df_finalS['Grupo'].apply(lambda x: grupo_counts[x])

# Eliminar filas con NaN en la columna "Título"
df_finalS = df_finalS.dropna(subset=['Título'])

# Crear una nueva columna "resautom" en el dataframe df_final.
df_finalS["resautom"] = ""

# Recorrer cada celda de la columna "Transcripción".
for index, row in df_finalS.iterrows():
    document = row["Transcripción"]

    # Utilizar expresiones regulares para buscar las oraciones relevantes.
    pattern = re.compile(r"(.*?[.!?])", re.IGNORECASE)
    relevant_sentences = re.findall(pattern, document)
    relevant_sentences = [sentence.strip() for sentence in relevant_sentences if "Semarnat" in sentence.upper() or "Secretaría de Medio Ambiente y Recursos Naturales" in sentence or "SEMARNAT" in sentence]

    # Unir las oraciones relevantes en una sola cadena de texto.
    summary_text = ". ".join(relevant_sentences)

    # Agregar el resultado a la columna "resautom" en el dataframe.
    df_finalS.at[index, "resautom"] = summary_text


################semarnat WORD INDICE##################

# Crear un documento de Word
document = Document()

# Establecer el tipo de letra y tamaño de fuente
font_name = "Montserrat"
font_size = Pt(12)
document.styles['Normal'].font.name = font_name
document.styles['Normal'].font.size = font_size

# Establecer los márgenes del documento como "Estrecho"
sections = document.sections
for section in sections:
    section.top_margin = section.bottom_margin = section.left_margin = section.right_margin = Pt(31.8)  # 0.44 pulgadas

# Establecer el interlineado
line_spacing = 1.5

# Agregar un encabezado al documento
#document.add_heading('Índice', level=1)
document.add_heading('SEMARNAT', level=1)

# Agrupar por "Grupo" y seleccionar la primera fila de cada grupo
df_grupos = df_finalS.groupby('Grupo').first()

# Ordenar el DataFrame por la columna "Notas" en orden descendente
df_grupos_sorted = df_grupos.sort_values(by='Notas', ascending=False)

# Crear una lista de todos los títulos
titulos_list = df_grupos_sorted['Título'].tolist()

# Agregar el listado de títulos como un índice al documento
#document.add_heading('Índice', level=1)
for i, titulo in enumerate(titulos_list, start=1):
    document.add_paragraph(f'{i}. {titulo}')

# Recorrer cada fila e imprimir los datos al documento
for _, fila in df_grupos_sorted.iterrows():
    # Obtener los datos de cada fila
    titulo = fila['Título']
    Resumen = fila['Resumen']
    URL = fila['URL del testigo']
    notas = fila['Notas']

    # Agregar el título y las notas al documento
    #document.add_paragraph(f'{titulo}')
    #document.add_paragraph(f'Notas: {notas}')
    #document.add_paragraph(f'Link: {URL}')

# Guardar el documento como un archivo de Word en la ruta especificada
document.save('/content/drive/MyDrive/profepa-sintesis/Excel/SÍNTESISS.docx')
print('Se ha creado el documento "SÍNTESIsS.docx" en la ruta especificada.')


Se ha creado el documento "SÍNTESIsS.docx" en la ruta especificada.


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [150]:
######profepa análisis###########
import os
import pandas as pd
import nltk
import re
from nltk.corpus import stopwords
from docx import Document
from docx.shared import Pt
from docx.enum.text import WD_PARAGRAPH_ALIGNMENT

# Ruta de la carpeta que contiene los archivos Excel
ruta = '/content/drive/MyDrive/profepa-sintesis/Excel/PROFEPA'

# Obtener la lista de archivos Excel en la carpeta
archivos_excel = [archivo for archivo in os.listdir(ruta) if archivo.endswith('.xlsx')]

# DataFrame final
df_final = pd.DataFrame()

# Recorrer los archivos Excel y cargarlos en el DataFrame final
for i, archivo in enumerate(archivos_excel):
    archivo_path = os.path.join(ruta, archivo)
    df = pd.read_excel(archivo_path)

    # Eliminar las filas correspondientes según el archivo
    if i == 0:
        df = df.iloc[1:]  # Eliminar la primera fila
    else:
        df = df.iloc[2:]  # Eliminar las tres primeras filas

    # Agregar el DataFrame al DataFrame final
    df_final = pd.concat([df_final, df], ignore_index=True)

# Utilizar la primera fila como cabeceras
df_final.columns = df_final.iloc[0]
df_final = df_final.iloc[1:]

# Convertimos las columnas 'Fecha' y 'Hora' a datetime
df_final['Fecha'] = pd.to_datetime(df_final['Fecha'])
df_final['Hora'] = pd.to_timedelta(df_final['Hora'])

# Creamos una nueva columna con la fecha y hora juntas
df_final['FechaHora'] = df_final['Fecha'] + df_final['Hora']

# Definimos la fecha y hora de inicio y fin
fecha_inicio = pd.to_datetime('2023-06-29 17:00:00')
fecha_fin = pd.to_datetime('2023-06-30 06:00:00')

# Filtramos el dataframe para que solo queden las filas dentro del rango de fecha y hora
df_final = df_final[(df_final['FechaHora'] >= fecha_inicio) & (df_final['FechaHora'] <= fecha_fin)]

#copiamos la columan titulo para trabajar sobre ella
df_final['Título_analisis'] = df_final['Título']

# Descargar los stopwords si no están descargados
nltk.download('stopwords')

# Obtener los stopwords en español
stopwords_esp = set(stopwords.words('spanish'))

# Función para eliminar palabras vacías y manejar valores nulos
def eliminar_palabras_vacias(texto):
    if isinstance(texto, str):  # Verificar si el valor no es nulo
        texto = re.sub(r'[^\w\s]', '', texto)  # Eliminar caracteres no deseados utilizando re.sub()
        texto = texto.strip()  # Eliminar espacios en blanco adicionales al inicio y al final
        palabras = texto.split()
        palabras_filtradas = [palabra for palabra in palabras if palabra not in stopwords_esp]
        return ' '.join(palabras_filtradas).lower()
    else:
        return texto

# Aplicar la función a la columna 'Título_analisis'
df_final['Título_analisis'] = df_final['Título_analisis'].apply(eliminar_palabras_vacias)

# Verificar y convertir a cadena los valores no nulos en la columna "Título_analisis"
df_final['Título_analisis'] = df_final['Título_analisis'].apply(lambda x: str(x) if pd.notnull(x) else '')

grupos = {}
for i in range(len(df_final)):
    titulo_i = df_final.iloc[i, df_final.columns.get_loc("Título_analisis")]
    if i not in grupos:
        grupo_actual = len(grupos) + 1
        grupos[i] = grupo_actual
    else:
        grupo_actual = grupos[i]
    for j in range(i+1, len(df_final)):
        titulo_j = df_final.iloc[j, df_final.columns.get_loc("Título_analisis")]
        titulo_j = str(titulo_j) if pd.notnull(titulo_j) else ''
        palabras_iguales = set(titulo_i.lower().split()) & set(titulo_j.lower().split())
        if len(palabras_iguales) >= 3:
            if j not in grupos:
                grupos[j] = grupo_actual
            else:
                grupo_j = grupos[j]
                if grupo_j != grupo_actual:
                    for k, v in grupos.items():
                        if v == grupo_j:
                            grupos[k] = grupo_actual

# Agregar una columna al DataFrame para mostrar los grupos
df_final['Grupo'] = [grupos[i] for i in range(len(df_final))]

grupo_counts = df_final['Grupo'].value_counts()
df_final['Notas'] = df_final['Grupo'].apply(lambda x: grupo_counts[x])

# Eliminar filas con NaN en la columna "Título"
df_final = df_final.dropna(subset=['Título'])

# Crear una nueva columna "resautom" en el dataframe df_final.
df_final["resautom"] = ""

# Recorrer cada celda de la columna "Transcripción".
for index, row in df_final.iterrows():
    document = row["Transcripción"]

    # Utilizar expresiones regulares para buscar las oraciones relevantes.
    pattern = re.compile(r"(.*?[.!?])", re.IGNORECASE)
    relevant_sentences = re.findall(pattern, document)
    relevant_sentences = [sentence.strip() for sentence in relevant_sentences if "Profepa" in sentence.upper() or "PROFEPA" in sentence or "Procuraduría Federal de Protección al Ambiente" in sentence]

    # Unir las oraciones relevantes en una sola cadena de texto.
    summary_text = ". ".join(relevant_sentences)

    # Agregar el resultado a la columna "resautom" en el dataframe.
    df_final.at[index, "resautom"] = summary_text





[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [151]:
#################WORD indice profepa##################

# Crear un documento de Word
document = Document()

# Establecer el tipo de letra y tamaño de fuente
font_name = "Montserrat"
font_size = Pt(12)
document.styles['Normal'].font.name = font_name
document.styles['Normal'].font.size = font_size

# Establecer los márgenes del documento como "Estrecho"
sections = document.sections
for section in sections:
    section.top_margin = section.bottom_margin = section.left_margin = section.right_margin = Pt(31.8)  # 0.44 pulgadas

# Establecer el interlineado
line_spacing = 1.5

# Agregar un encabezado al documento
document.add_heading('Índice', level=1)
document.add_heading('PROFEPA', level=1)

# Agrupar por "Grupo" y seleccionar la primera fila de cada grupo
df_grupos = df_final.groupby('Grupo').first()

# Ordenar el DataFrame por la columna "Notas" en orden descendente
df_grupos_sorted = df_grupos.sort_values(by='Notas', ascending=False)

# Crear una lista de todos los títulos
titulos_list = df_grupos_sorted['Título'].tolist()

# Agregar el listado de títulos como un índice al documento
#document.add_heading('Índice', level=1)
for i, titulo in enumerate(titulos_list, start=1):
    document.add_paragraph(f'{i}. {titulo}')

# Recorrer cada fila e imprimir los datos al documento
for _, fila in df_grupos_sorted.iterrows():
    # Obtener los datos de cada fila
    titulo = fila['Título']
    Resumen = fila['Resumen']
    URL = fila['URL del testigo']
    notas = fila['Notas']
    resautom = fila['resautom']

   # Agregar el título y las notas al documento
    #document.add_heading(titulo, level=1)
    #document.add_paragraph(f'Notas: {notas}')
    #document.add_paragraph()
    #document.add_paragraph(resautom)
    #document.add_paragraph(f'Nota: {URL}')

# Guardar el documento como un archivo de Word en la ruta especificada
document.save('/content/drive/MyDrive/profepa-sintesis/Excel/SÍNTESISP.docx')
print('Se ha creado el documento "SÍNTESIsP.docx" en la ruta especificada.')

Se ha creado el documento "SÍNTESIsP.docx" en la ruta especificada.


In [152]:
# PROFEPA Crear un documento de Word completo de notas sin indice############################
document = Document()

# Establecer el tipo de letra y tamaño de fuente
font_name = "Montserrat"
font_size = Pt(12)
document.styles['Normal'].font.name = font_name
document.styles['Normal'].font.size = font_size

# Establecer los márgenes del documento como "Estrecho"
sections = document.sections
for section in sections:
    section.top_margin = section.bottom_margin = section.left_margin = section.right_margin = Pt(31.8)  # 0.44 pulgadas

# Establecer el interlineado
line_spacing = 1.5

# Agregar un encabezado al documento
#document.add_heading('Índice', level=1)
document.add_heading('PROFEPA', level=1)

# Agrupar por "Grupo" y seleccionar la primera fila de cada grupo
df_grupos = df_final.groupby('Grupo').first()

# Ordenar el DataFrame por la columna "Notas" en orden descendente
df_grupos_sorted = df_grupos.sort_values(by='Notas', ascending=False)

# Crear una lista de todos los títulos
titulos_list = df_grupos_sorted['Título'].tolist()

# Recorrer cada fila e imprimir los datos al documento
for _, fila in df_grupos_sorted.iterrows():
    # Obtener los datos de cada fila
    titulo = fila['Título']
    Resumen = fila['Resumen']
    URL = fila['URL del testigo']
    notas = fila['Notas']
    resautom = fila['resautom']

   # Agregar el título y las notas al documento
    document.add_heading(titulo, level=1)
    #document.add_paragraph(f'Notas: {notas}')
    document.add_paragraph()
    document.add_paragraph(resautom)
    document.add_paragraph(f'Nota: {URL}')

# Guardar el documento como un archivo de Word en la ruta especificada
document.save('/content/drive/MyDrive/profepa-sintesis/Excel/SÍNTESISPC.docx')
print('Se ha creado el documento "SÍNTESIsP.docx" en la ruta especificada.')

Se ha creado el documento "SÍNTESIsP.docx" en la ruta especificada.


In [153]:
#####3semarnat completo#####

from docx import Document
from docx.shared import Pt

# Crear un documento de Word SOLO NOTAS
document = Document()

# Establecer el tipo de letra y tamaño de fuente
font_name = "Montserrat"
font_size = Pt(12)
document.styles['Normal'].font.name = font_name
document.styles['Normal'].font.size = font_size

# Establecer los márgenes del documento como "Estrecho"
sections = document.sections
for section in sections:
    section.top_margin = section.bottom_margin = section.left_margin = section.right_margin = Pt(31.8)  # 0.44 pulgadas

# Establecer el interlineado
line_spacing = 1.5

# Agregar un encabezado al documento
document.add_heading('SEMARNAT', level=1)

# Agrupar por "Grupo" y seleccionar la primera fila de cada grupo
df_grupos = df_finalS.groupby('Grupo').first()

# Ordenar el DataFrame por la columna "Notas" en orden descendente
df_grupos_sorted = df_grupos.sort_values(by='Notas', ascending=False)

# Crear una lista de todos los títulos
titulos_list = df_grupos_sorted['Título'].tolist()

# Recorrer cada fila e imprimir los datos al documento
for _, fila in df_grupos_sorted.iterrows():
    # Obtener los datos de cada fila
    titulo = fila['Título']
    Resumen = fila['Resumen']
    URL = fila['URL del testigo']
    notas = fila['Notas']
    resautom = fila['resautom']

    # Agregar el título y las notas al documento
    document.add_heading(titulo, level=1)
    document.add_paragraph()
    document.add_paragraph(resautom)
    document.add_paragraph(f'Nota: {URL}')

# Guardar el documento como un archivo de Word en la ruta especificada
document.save('/content/drive/MyDrive/profepa-sintesis/Excel/SÍNTESISSC.docx')
print('Se ha creado el documento "SÍNTESISPC.docx" en la ruta especificada.')


Se ha creado el documento "SÍNTESISPC.docx" en la ruta especificada.


In [154]:
########DOCUMENTO FINAL###################
import os
import random
from PIL import Image
from docx import Document
from docx.shared import Inches

# Ruta de la carpeta de imágenes JPG
ruta_imagenes_jpg = "/content/drive/MyDrive/Procuraduria/COMUNICACION SOCIAL/C.S. - PROFEPA/Diseño/Portadas-Síntesis/matjpg"

# Obtener la lista de imágenes JPG en la carpeta
lista_imagenes_jpg = [imagen for imagen in os.listdir(ruta_imagenes_jpg) if imagen.lower().endswith(".jpg")]

# Elegir una imagen al azar de la lista
imagen_elegida_jpg = random.choice(lista_imagenes_jpg)

# Ruta completa de la imagen JPG elegida
ruta_imagen_elegida_jpg = os.path.join(ruta_imagenes_jpg, imagen_elegida_jpg)

# Ruta de destino de la imagen PNG convertida
ruta_imagen_convertida_png = "/content/drive/MyDrive/profepa-sintesis/Excel/imagen.png"

# Abrir la imagen JPG con Pillow
imagen_pil = Image.open(ruta_imagen_elegida_jpg)

# Convertir la imagen a modo RGB
imagen_rgb = imagen_pil.convert("RGB")

# Guardar la imagen en formato PNG en la ruta de destino
imagen_rgb.save(ruta_imagen_convertida_png, "PNG")

# Ruta de destino del documento de Word
ruta_documento_final = "/content/drive/MyDrive/profepa-sintesis/Excel/documento_word.docx"

# Crear un documento final de Word
doc_final = Document()

# Ajustar la imagen a un tamaño específico en pulgadas
width = Inches(6.5)
height = Inches(9)

# Insertar la imagen en la primera página del documento final con el tamaño ajustado
doc_final.add_picture(ruta_imagen_convertida_png, width=width, height=height)

# Abrir y agregar la información del primer documento
ruta_informacion1 = "/content/drive/MyDrive/profepa-sintesis/Excel/SÍNTESISP.docx"
doc_informacion1 = Document(ruta_informacion1)
for element in doc_informacion1.element.body:
    doc_final.element.body.append(element)

# Abrir y agregar la información del segundo documento
ruta_informacion2 = "/content/drive/MyDrive/profepa-sintesis/Excel/SÍNTESISS.docx"
doc_informacion2 = Document(ruta_informacion2)
for element in doc_informacion2.element.body:
    doc_final.element.body.append(element)

# Abrir y agregar la información del tercer documento
ruta_informacion3 = "/content/drive/MyDrive/profepa-sintesis/Excel/SÍNTESISPC.docx"
doc_informacion3 = Document(ruta_informacion3)
for element in doc_informacion3.element.body:
    doc_final.element.body.append(element)

# Abrir y agregar la información del cuarto documento
ruta_informacion4 = "/content/drive/MyDrive/profepa-sintesis/Excel/SÍNTESISSC.docx"
doc_informacion4 = Document(ruta_informacion4)
for element in doc_informacion4.element.body:
    doc_final.element.body.append(element)

# Guardar el documento final con la imagen y la información
doc_final.save(ruta_documento_final)

print("SÍNTESIS TERMINADA")


SÍNTESIS TERMINADA
